In [7]:
import numpy as np
import re
import random

In [11]:
#structure of intent_form = INTENT_NAME(OUTPUT_SYMBOL or None, INPUTSymbols,......)

intent_forms = ["IDENTIFY(T,'orange')",
               "ASK_CLARIFICATION(None, T) AND IDENTIFY(T,'greyish orange')",
               "CONFIRMATION(None, 'YES')",
               "IDENTIFY(T, 'purple')",
               "ASK_CLARIFICATION(T, 'light purple') AND IDENTIFY(T, 'light purple')",
               "IDENTIFY_FROM_SET(T, S, 'brighter') AND IDENTIFY_SET(S,'dark purple')"]

INTENTS = ['IDENTIFY', 'ASK_CLARIFICATION', 'CONFIRMATION', 'IDENTIFY_FROM_SET', 'IDENTIFY_SET']

In [2]:
#this will be needed at a later stage
#there could be the case that intents are not ordered and we have to do some search to resolve inter-dependenices
#sort according to dependency resolution will help later
#e.g. c -> b; a -> b,c; b-> constant(x) should be ordered like this:
#b -> constant(x); c -> b; a -> b, c
def sort_dependencies(intent_arg_map):
    pass

In [3]:
def invoke_listener_model(color):
    #this will return probability distribution for all the color patches given the color term
    #and is just a place holder for actual brian's code
    return [random.random() for i in range(3)]

def invoke_speaker_model(color):
    vocab_size = 829
    return [[random.random() for i in range(vocab_size)] for i in range(3)]

In [4]:
#this part is what Shengjie will be working at!
#keep the pattern for function names like this
#eval_INTENT_NAME(args...)
#All of these functions contain code which should just convey an idea of what is going to be the structure
#so it is easier for shengjie to start and me to correct my mistakes while defining this structure
def eval_IDENTIFY(color_name):
    return invoke_listener_model(color_name)

def eval_IDENTIFY_SET(color_term):
    return_set = set()
    prob_term_given_patch = invoke_speaker_model(color_term)
    for i in range(3):
        #here the condition should be to check if color term is within topk of the given colors terms
        if random.randint(0,1) == 1: 
            return_set.add(i)
    return return_set

def eval_ASK_CLARIFICATION(target):
    #this just returns but server the purpose of conveying that relation should be ASK_CLARIFICATION
    return (target, 'ASK_CLARIFICATION')

def eval_CONFIRMATION(intent):
    #serves similar purpose to ASK_CLARIFICATION. THE RELATION WOULD BE CONFIRMATION IN THIS CASE, though.
    return (intent, 'CONFIRM')

def eval_IDENTIFY_FROM_SET(patch_set, prop):
    #this function should have a structure like this:
    #if system understands prop:
    #    return (identified_color_patch_from_set, None)
    #else:
    #    return (same_patches_as_received, prop) conveying the sense that system needs to ask a clarification about what
    # this property means
    return patch_set


In [9]:
def evaluate(composition):
    pattern = '(.+)(\()((\'?[\s\w]*\'?,\s?)*)(\'?[\s\w]*\'?)(\))'
    prog = re.compile(pattern)
    output = dict()
    evaluation = None
    for i, intent in enumerate(composition.split('AND')[::-1]):
        intent = intent.strip()
        groups = None
        try:
            groups = prog.search(intent).groups()
        except Exception as e:
            print('Illegal Intent Format:', intent)
            return
        
        name = groups[0]
        argument_symbols = []
        for i, args in enumerate(groups[2:-1]):
            if i == 1:
                continue
            argument_symbols+=[sym.strip() for sym in args.split(',')]
        argument_symbols.remove('')
        
        arguments = [arg if arg[0] == "'" else output[arg] for arg in argument_symbols[1:]]
        if argument_symbols[0] != 'None':
            output[argument_symbols[0]] = eval('eval_' + name)(*arguments)
            evaluation = output[argument_symbols[0]]
        else:
            evaluation = eval('eval_' + name)(*arguments)
    return evaluation

In [12]:
for intent_composition in intent_forms:
    extracted_features = evaluate(intent_composition)
    print(extracted_features)
    

[0.8216388598534146, 0.6332709852462626, 0.37178968253750555]
([0.12264169580711215, 0.9162553335079625, 0.9498327438870223], 'ASK_CLARIFICATION')
("'YES'", 'CONFIRM')
[0.5181628347332321, 0.31112805263490295, 0.1709863767101536]
("'light purple'", 'ASK_CLARIFICATION')
set()
